In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import gc

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
%matplotlib inline

!pip install koreanize-matplotlib
import koreanize_matplotlib

In [28]:
pip install factor_analyzer

In [29]:
import sys
sys.path.append("/content/drive/MyDrive/week_hands_on_project5/whiskey/model")

from recommender_model import WhiskyRecommender

In [30]:
import pickle

with open("/content/drive/MyDrive/week_hands_on_project5/whiskey/model/whisky_recommender.pkl", "rb") as f:
    recommender = pickle.load(f)

df = recommender.df

# 평가

## 𝟭. FA space 평가

1) 전반적인 수치 / 공통성

      •	KMO = 0.51, Bartlett p < 0.001로 요인분석 적용 가능성을 확인하였다.

      •	두 요인은 Body 지표(communality 0.89)를 매우 잘 설명하며,
    Richness/Smoke/Sweetness는 0.30~0.38 수준으로 큰 방향성을 중심으로 요약된다.


    ⏺ 따라서 본 프로젝트에서는 **Body·Richness를 중심으로 한 전반적 강도 축(FA1)**과
    Smoke–Sweetness 대비 축(FA2) 두 개의 축으로 taste space를 정의하고,
    세밀한 단맛·훈연 강도보다는 “대략적인 포지션”을 기준으로 추천/진열 전략을 설계하였다.

2. 각 style 지표가 정상적으로 FA 에서 적용되고 있나?
    - Body 태그별 FA1 평균은 light < medium < full 순으로 단조 증가하여,
    FA1이 전반적인 바디감·무게감을 설명하는 1차 요인임을 확인하였다.
    - Richness 태그에서도 lean < round < rich 순으로 FA1이 증가하여,
    FA1이 바디뿐 아니라 전반적인 농도/점성(richness)의 강약을 함께 반영하는 축임을 확인하였다.
    - Smoke 태그별 FA2 평균은 non < light_smoke < smoky 순으로 뚜렷하게 증가하여,
    FA2가 위스키의 스모키함·peaty 특성을 설명하는 주요 요인임을 확인하였다.
    - Sweetness 태그에서는 sweet < balanced < dry 순으로 FA2가 증가하여,
    FA2가 ‘스모키함 ↔ 단맛’의 대비 축을 동시에 설명하는 구조임을 확인하였다.

In [31]:
from factor_analyzer.factor_analyzer import calculate_kmo, calculate_bartlett_sphericity

def evaluate_fa_space(
    recommender,
    style_cols=None,
    tag_cols=None,
    pattern_col="flavour_pattern",
):
    """
    1) KMO, Bartlett
    2) 공통성(communality)
    3) tag / flavour_pattern과 F1,F2의 관계 요약
    """

    if style_cols is None:
        style_cols = ["style_body", "style_richness", "style_smoke", "style_sweetness"]
    if tag_cols is None:
        tag_cols = ["body_tag", "richness_tag", "smoke_tag", "sweetness_tag"]

    df_taste = recommender.df_taste.copy()
    X = df_taste[style_cols].dropna()

    # --- 1) KMO, Bartlett ---
    kmo_all, kmo_model = calculate_kmo(X)
    chi_square, bartlett_p = calculate_bartlett_sphericity(X)

    # --- 2) Communalities (공통성) ---
    fa = recommender.fa_model
    communalities = fa.get_communalities()
    comm_df = pd.DataFrame(
        {"variable": style_cols, "communality": communalities}
    )

    # --- 3) Tag / pattern별 F1,F2 평균 ---
    fa_scores = df_taste[["FA1", "FA2"]].copy()

    tag_summary = {}
    for col in tag_cols:
        if col in df_taste.columns:
            tmp = df_taste[[col, "FA1", "FA2"]].dropna()
            tag_summary[col] = (
                tmp.groupby(col)[["FA1", "FA2"]]
                .mean()
                .reset_index()
            )

    pattern_summary = None
    if pattern_col in df_taste.columns:
        pattern_summary = (
            df_taste[[pattern_col, "FA1", "FA2"]]
            .groupby(pattern_col)[["FA1", "FA2"]]
            .mean()
            .reset_index()
        )

    result = {
        "kmo_model": float(kmo_model),
        "bartlett_chi2": float(chi_square),
        "bartlett_p": float(bartlett_p),
        "communalities": comm_df,
        "tag_summary": tag_summary,
        "pattern_summary": pattern_summary,
    }
    return result


In [32]:
# 모델 2. FA taste space
fa_eval = evaluate_fa_space(recommender)

print("1) 전반적인 수치")
display(fa_eval["kmo_model"], fa_eval["bartlett_p"])


print("\n\n 2) 변수별 공통성")
display(fa_eval["communalities"])

print("\n\n 3) 태그 기준 F1/F2 평균")
display(fa_eval["tag_summary"]["body_tag"])
display(fa_eval["tag_summary"]["richness_tag"])
display(fa_eval["tag_summary"]["smoke_tag"])
display(fa_eval["tag_summary"]["sweetness_tag"])

print("\n\n4) flavour_pattern 기준 F1/F2 평균")
display(fa_eval["pattern_summary"].head())

1) 전반적인 수치


0.5082555964041512

2.4201150263240584e-65



 2) 변수별 공통성


,variable,communality
0,style_body,0.886935
1,style_richness,0.348347
2,style_smoke,0.380003
3,style_sweetness,0.297633


...

,richness_tag,FA1,FA2
0,lean,-1.189141,-0.082569
1,rich,0.684774,-0.111230
2,round,-0.212325,0.064646


,smoke_tag,FA1,FA2
0,light_smoke,-0.063723,0.502280
1,non,-0.027018,-0.338749
2,smoky,0.303272,1.489053


,sweetness_tag,FA1,FA2
0,balanced,-0.037187,0.120027
1,dry,-0.337806,1.913644
2,sweet,0.213653,-0.782805




4) flavour_pattern 기준 F1/F2 평균


,flavour_pattern,FA1,FA2
0,full_rich_light_smoke_balanced,1.321091,0.448034
1,full_rich_light_smoke_sweet,1.254881,-0.157900
2,full_rich_non_balanced,1.206156,-0.252192
3,full_rich_non_sweet,1.239348,-0.874208
4,full_rich_smoky_balanced,1.271981,1.486752


## 𝟮. 거리 기반 성능 검증


**<<Taste KNN은 Random 추천 대비 40배 이상 정교하다>>**


Taste KNN의 평균 거리(0.034)는 Random baseline(1.42)에 비해
약 40배 이상 작은 수준으로 나타났다.
일부 샘플에서는 KNN 이웃의 거리가 0 수준에 가까워,
improvement_ratio는 사실상 무한대(inf)에 수렴하였다.

이는 FA 기반 Taste Space가 위스키 맛 구조를 매우 안정적으로 분리하며,
Taste 기반 추천이 무작위 추천 대비 압도적으로 우수함을 의미한다.

In [33]:
def evaluate_taste_knn_vs_random(recommender, n_seeds=30, k=10, random_state=42):
    rng = np.random.default_rng(random_state)
    df = recommender.df

    seeds = df[df["style_missing"] == 0]["name"].dropna().unique()
    if len(seeds) < n_seeds:
        n_seeds = len(seeds)
    seed_names = rng.choice(seeds, size=n_seeds, replace=False)

    records = []

    for name in seed_names:
        idx = recommender._get_index_by_name(name)
        fa_point = df.loc[idx, ["FA1", "FA2"]].values.astype(float)

        # 1) Taste KNN
        knn_df = recommender._knn_taste_point(fa_point, top_k=k)
        knn_mean_dist = knn_df["taste_distance"].mean()

        # 2) 랜덤 이웃
        candidates = df[(df.index != idx) & df["FA1"].notna() & df["FA2"].notna()]
        random_sample = candidates.sample(
            n=min(k, len(candidates)),
            replace=False,
            random_state=int(rng.integers(1e9)),
        )
        rand_dist = np.sqrt(
            (random_sample["FA1"].values - fa_point[0]) ** 2
            + (random_sample["FA2"].values - fa_point[1]) ** 2
        )
        rand_mean_dist = rand_dist.mean()

        records.append(
            {
                "seed": name,
                "knn_mean_dist": knn_mean_dist,
                "random_mean_dist": rand_mean_dist,
            }
        )

    result = pd.DataFrame(records)
    result["improvement_ratio"] = result["random_mean_dist"] / result["knn_mean_dist"]
    return result

In [34]:
# taste knn vs 랜덤 거리 비교
taste_eval = evaluate_taste_knn_vs_random(recommender, n_seeds=50, k=10)
taste_eval[["knn_mean_dist", "random_mean_dist", "improvement_ratio"]].mean()

,0
knn_mean_dist,0.034001
random_mean_dist,1.422613
improvement_ratio,inf


## 𝟯. 제품 노출 편향

    지금 coverage_ratio는 모델 자체가 제공하는 추천 다양성이 부족하다는 신호이며,
    taste/meta 구조의 편향이 그대로 드러난 결과다.
    추천 시스템 품질, 사용자 경험, 비즈니스 적합성 측면에서
    분명 개선이 필요한 지표다.

In [35]:
def evaluate_business_exposure(
    recommender,
    n_style_seeds=50,
    n_nostyle_seeds=30,
    n_survey_seeds=20,
    top_k_taste=20,
    top_k_meta=20,
    random_state=42,
):
    rng = np.random.default_rng(random_state)
    df = recommender.df

    all_reco_names = []

    # --- 1) 스타일 있는 제품 seed (Scenario A) ---
    style_seeds = (
        df[df["style_missing"] == 0]["name"].dropna().unique()
    )
    style_seeds = rng.choice(
        style_seeds, size=min(n_style_seeds, len(style_seeds)), replace=False
    )

    for name in style_seeds:
        res = recommender.recommend(
            product_list=[name],
            top_k_taste=top_k_taste,
            top_k_meta=top_k_meta,
        )
        for key in ["taste_based_main", "meta_based_main"]:
            if key in res and isinstance(res[key], pd.DataFrame):
                all_reco_names.extend(res[key]["name"].tolist())

    # --- 2) 스타일 없는 제품 seed (Scenario B) ---
    nostyle_seeds = (
        df[df["style_missing"] == 1]["name"].dropna().unique()
    )
    if len(nostyle_seeds) > 0:
        nostyle_seeds = rng.choice(
            nostyle_seeds, size=min(n_nostyle_seeds, len(nostyle_seeds)), replace=False
        )
        for name in nostyle_seeds:
            res = recommender.recommend(
                product_list=[name],
                top_k_meta=top_k_meta,
            )
            if "meta_based_main" in res:
                all_reco_names.extend(res["meta_based_main"]["name"].tolist())

    # --- 3) 설문 시드 (Scenario C) ---
    body_levels = ["light", "medium", "full"]
    rich_levels = ["lean", "round", "rich"]
    smoke_levels = ["non", "light-smoky", "smoky"]
    sweet_levels = ["dry", "balanced", "sweet"]

    for _ in range(n_survey_seeds):
        res = recommender.recommend(
            body_level=rng.choice(body_levels),
            rich_level=rng.choice(rich_levels),
            smoke_level=rng.choice(smoke_levels),
            sweet_level=rng.choice(sweet_levels),
            selected_families=None,
            top_k_taste=top_k_taste,
            top_k_meta=top_k_meta,
        )
        for key in ["taste_only_main", "meta_expansion"]:
            if key in res and isinstance(res[key], pd.DataFrame):
                all_reco_names.extend(res[key]["name"].tolist())

    # -----------------------------
    #  노출 통계 계산
    # -----------------------------
    exposure_counts = (
        pd.Series(all_reco_names, name="name").value_counts().rename("exposure_count")
    )
    exposure_df = (
        exposure_counts.to_frame()
        .merge(df[["name", "whisky_type", "region", "price_bin", "style_missing"]],
               left_index=True, right_on="name", how="left")
    )

    total_items = df["name"].nunique()
    covered_items = exposure_counts.index.nunique()
    coverage_ratio = covered_items / total_items

    # 세그먼트별 share
    seg_cols = ["whisky_type", "region", "price_bin", "style_missing"]
    seg_exposure = (
        exposure_df.groupby(seg_cols)["name"]
        .count()
        .rename("exposure_items")
        .reset_index()
    )
    seg_exposure["exposure_share"] = seg_exposure["exposure_items"] / seg_exposure["exposure_items"].sum()

    inventory_seg = (
        df.groupby(seg_cols)["name"]
        .nunique()
        .rename("inventory_items")
        .reset_index()
    )
    inventory_seg["inventory_share"] = inventory_seg["inventory_items"] / inventory_seg["inventory_items"].sum()

    seg_merged = seg_exposure.merge(
        inventory_seg,
        on=seg_cols,
        how="outer",
    ).fillna(0)

    return {
        "coverage_ratio": coverage_ratio,
        "exposure_df": exposure_df,
        "segment_exposure": seg_merged,
    }

In [36]:
# 노출 편향
biz_eval = evaluate_business_exposure(recommender)
print(f"==== {biz_eval['coverage_ratio']*100:.2f}% 제품이 추천 결과 어딘가에서 최소 1번 등장 했다. ====\n")
print("==== 아예 추천에 한 번도 등장하지 않은 제품 ====\n")

exposure_df = biz_eval["exposure_df"]

# 등장한 제품들
appeared = set(exposure_df["name"].unique())

# 전체 제품들
all_items = set(df["name"].unique())

# 노출되지 않은 제품 리스트
not_appeared = sorted(all_items - appeared)

print("노출되지 않은 제품 개수:", len(not_appeared))
print("예시 20개:")
for name in not_appeared[:20]:
    print("-", name)

==== 48.34% 제품이 추천 결과 어딘가에서 최소 1번 등장 했다. ====

==== 아예 추천에 한 번도 등장하지 않은 제품 ====

노출되지 않은 제품 개수: 546
예시 20개:
- 20 Year Old Speyside Cognac FinishWhisky Works
- A Good Old-Fashioned Christmas Whisky 15 Year Old2024 Edition
- Aberlour 10 Year OldBot.1980s
- Aberlour 15 Year OldCuvée Marie d'Ecosse
- Aberlour 18 Year Old
- Aberlour 196921 Year Old Bot.1991 Sherry Cask
- Aberlour 1990Litre
- Aberlour 5 Year OldBot.1990s Decanter Gift Pack
- Aberlour A'Bunadh Batch 57
- Aberlour A'Bunadh Batch 59
- Aberlour A'bunadh Batch 16
- Ainslie's Royal EdinburghBot.1960s
- Akashi 5 Year OldRed Wine Cask 61592
- Akashi Blended Sherry Cask Finish
- Allt A Bhainne 199628 Year Old Thompson Bros
- Allt-A-Bhainne 200618 Year Old Cask #16601402 Connoisseurs Choice
- Allt-a-Bhainne 19929 Year Old Cadenhead's
- Allt-a-Bhainne 200021 Year Old Sherry Cask Signatory
- Anchor BlendBot.1960s
- Antiquary 30 Year OldSherry Cask Finish


In [37]:
#추천에 많이 등장한 TOP 제품들
top_exposed = (
    exposure_df.sort_values("exposure_count", ascending=False)
               .head(20)
)
top_exposed

,exposure_count,name,whisky_type,region,price_bin,style_missing
859,22,The Six Isles Batch Strength,blended malt,unknown_scotland,mid,0
19,19,Ballantine's Finest,blended,unknown_scotland,low,0
829,19,Monkey Shoulder Smokey Monkey,blended malt,unknown_scotland,low,0
567,17,Spey Fumare,single malt,speyside,mid,0
950,17,Highland Harvest7 Casks Organic Blended Malt W...,blended malt,unknown_scotland,low,0
738,17,Yoichi Single Malt,single malt,unknown_japan,mid,0
18,17,Ballantine's Finest True Music Icons Elton Joh...,blended,unknown_scotland,low,0
395,16,Aberlour 12 Year Old Double CaskGift Box,single malt,speyside,mid,0
886,15,The Guard Bridge Blended Malt,blended malt,unknown_scotland,mid,0
21,15,Johnnie Walker Double Black,blended,unknown_scotland,low,0


In [38]:
#  추천에 거의 등장하지 않은 BOTTOM 제품들
bottom_exposed = (
    exposure_df.sort_values("exposure_count", ascending=True)
               .head(20)
)
bottom_exposed

,exposure_count,name,whisky_type,region,price_bin,style_missing
470,1,Tomintoul 10 Year Old,single malt,unknown_scotland,low,0
336,1,Jack Daniel's Bonded100 Proof Bottled-in-Bond,tennessee,tennessee,mid,0
178,1,Pure Scot Virgin Oak,blended,unknown_scotland,mid,0
922,1,Glenborrodale Batch 811 Year Old Adelphi,blended malt,unknown_scotland,mid,0
438,1,Glenlivet 15 Year OldFrench Oak Reserve,single malt,speyside,mid,0
121,1,Black & WhiteBot.1970s,blended,unknown_scotland,high,1
844,1,Johnnie Walker Black Label 12 Year Old Islay O...,blended malt,islay,low,1
896,1,Westport 200418 Year Old Sherry Cask Wilson & ...,blended malt,unknown_scotland,high,0
379,1,Very Old Barton 100 Proof Bourbon,bourbon,kentucky,low,1
754,1,The House of Suntory Triology Pack3x20cl,other,unknown_japan,mid,1


### 𝟯-𝟭 특징 분석

**프리미엄 및 희귀 제품 전용 (높은 연식, 높은 가격)의 위스키는 따로 취급할 필요가 있다.**

제안 해결 방안

    유저가 받은 메인 추천 리스트를 기준으로
    그 주변에 있는 ‘가까운 친구들’ 중에서
    아직 거의 안 노출되던 애들을 골라서
    Something else / 다른 선택지 슬롯으로 7개 정도 추가로 보여주기

In [39]:
exposure_df = biz_eval["exposure_df"]   # 추천에 실제로 등장한 상품들
df = recommender.df.copy()

# 이 name이 추천 리스트에 한 번이라도 출현했는지 여부
df["exposed"] = df["name"].isin(exposure_df["name"])

exposed_df     = df[df["exposed"] == True]
not_exposed_df = df[df["exposed"] == False]

print("전체 상품 수:", len(df))
print("노출된 상품 수:", exposed_df["name"].nunique())
print("노출 안 된 상품 수:", not_exposed_df["name"].nunique())

전체 상품 수: 1083
노출된 상품 수: 511
노출 안 된 상품 수: 546


In [40]:

def exposure_by_feature(df, col):
    """
    col 기준으로
    - 그 값에 속한 상품 수(total_items)
    - 노출된 상품 수(exposed_items)
    - 노출 안 된 상품 수(not_exposed_items)
    - 노출률(exposure_rate = exposed_items / total_items)
    를 계산
    """
    # 각 그룹별 전체 아이템 수
    total = (
        df.groupby(col)["name"]
          .nunique()
          .rename("total_items")
    )

    # 노출 / 미노출 카운트
    exp = (
        df.groupby(["exposed", col])["name"]
          .nunique()
          .rename("items")
          .reset_index()
    )

    # wide 형태로 피벗
    exp_pivot = (
        exp.pivot(index=col, columns="exposed", values="items")
           .rename(columns={False: "not_exposed_items", True: "exposed_items"})
           .fillna(0)
    )

    out = (
        total.to_frame()
             .join(exp_pivot, how="left")
             .fillna(0)
             .reset_index()
    )
    out["exposure_rate"] = out["exposed_items"] / out["total_items"].replace(0, np.nan)

    # 노출률 낮은 값부터 보고 싶으면 sort_values 사용
    out = out.sort_values("exposure_rate")
    return out
cols_to_check = ["whisky_type", "region", "price_bin", "style_missing", "age_bin"]

for c in cols_to_check:
    print(f"\n==== {c} 기준 노출 편향 ====")
    display(exposure_by_feature(df, c))


==== whisky_type 기준 노출 편향 ====


,whisky_type,total_items,not_exposed_items,exposed_items,exposure_rate
3,corn,1,1.0,0.0,0.000000
7,single malt,446,276.0,170.0,0.381166
4,grain,33,20.0,13.0,0.393939
5,other,12,7.0,5.0,0.416667
0,blended,325,189.0,136.0,0.418462
8,single pot still,32,8.0,24.0,0.750000
1,blended malt,138,34.0,104.0,0.753623
2,bourbon,45,9.0,36.0,0.800000
9,tennessee,11,2.0,9.0,0.818182
6,rye,14,0.0,14.0,1.000000



==== region 기준 노출 편향 ====


,region,total_items,not_exposed_items,exposed_items,exposure_rate
6,lowland,10,7.0,3.0,0.300000
12,unknown_japan,95,64.0,31.0,0.326316
9,speyside,341,211.0,130.0,0.381232
13,unknown_scotland,386,203.0,183.0,0.474093
14,unknown_usa,7,3.0,4.0,0.571429
3,island,5,2.0,3.0,0.600000
11,unknown_ireland,120,43.0,77.0,0.641667
0,campbeltown,6,2.0,4.0,0.666667
10,tennessee,11,2.0,9.0,0.818182
5,kentucky,48,7.0,41.0,0.854167



==== price_bin 기준 노출 편향 ====


,price_bin,total_items,not_exposed_items,exposed_items,exposure_rate
2,luxury,178,167,11,0.061798
4,premium,212,197,15,0.070755
0,high,184,85,99,0.538043
3,mid,336,81,255,0.758929
1,low,155,17,138,0.890323



==== style_missing 기준 노출 편향 ====


,style_missing,total_items,not_exposed_items,exposed_items,exposure_rate
1,1,440,286,154,0.35000
0,0,618,261,357,0.57767



==== age_bin 기준 노출 편향 ====


,age_bin,total_items,not_exposed_items,exposed_items,exposure_rate
3,22+,87,87.0,0.0,0.000000
2,17-21,100,89.0,11.0,0.110000
1,13-16,97,57.0,40.0,0.412371
0,0-8,53,30.0,23.0,0.433962
4,9-12,177,78.0,99.0,0.559322
5,Unknown,543,205.0,338.0,0.622468


## 𝟰. 모델 안정성

    이 taste-based 추천 모델은 데이터 기반으로 완전 결정론적 구조를 가지기 때문에,
    운영 중 재학습 혹은 인덱스 정렬 변경이 발생해도 추천 결과가 흔들리지 않는 높은 안정성을 보인다.

    이는 실제 제품 비교/추천 서비스에서 매우 중요한 신뢰성과 일관성을 확보했다는 것을 의미한다.



In [41]:
def evaluate_model_stability(
    df,
    style_cols=None,
    n_seeds=30,
    k=10,
    random_state=42,
):
    """
    같은 데이터로 recommender를 두 번 학습했을 때
    - FA loadings 일관성
    - Taste 추천 Top-k 겹침 정도
    를 측정
    """
    if style_cols is None:
        style_cols = ["style_body", "style_richness", "style_smoke", "style_sweetness"]

    from recommender_model import WhiskyRecommender  # 네 모듈 이름 그대로 사용

    # 1) 모델 두 번 학습
    rec1 = WhiskyRecommender(df)
    rec2 = WhiskyRecommender(df)

    # --- 1) FA loadings 유사도 ---
    load1 = pd.DataFrame(
        rec1.fa_model.loadings_,
        index=style_cols,
        columns=["F1", "F2"],
    )
    load2 = pd.DataFrame(
        rec2.fa_model.loadings_,
        index=style_cols,
        columns=["F1", "F2"],
    )

    corr_F1 = np.corrcoef(load1["F1"], load2["F1"])[0, 1]
    corr_F2 = np.corrcoef(load1["F2"], load2["F2"])[0, 1]

    # --- 2) Taste 추천 Top-k 겹침 (Jaccard) ---
    rng = np.random.default_rng(random_state)
    df_all = df[df["style_missing"] == 0]
    seeds = df_all["name"].dropna().unique()
    if len(seeds) < n_seeds:
        n_seeds = len(seeds)
    seed_names = rng.choice(seeds, size=n_seeds, replace=False)

    overlaps = []

    for name in seed_names:
        # meta 관련 파라미터는 빼고, taste만 지정
        res1 = rec1.recommend(product_list=[name], top_k_taste=k)
        res2 = rec2.recommend(product_list=[name], top_k_taste=k)

        # taste_based_main이 없거나 비어있으면 스킵
        if "taste_based_main" not in res1 or "taste_based_main" not in res2:
            continue

        set1 = set(res1["taste_based_main"]["name"].head(k))
        set2 = set(res2["taste_based_main"]["name"].head(k))

        if set1 or set2:
            jaccard = len(set1 & set2) / len(set1 | set2)
            overlaps.append(jaccard)

    stability = {
        "fa_loading_corr_F1": float(corr_F1),
        "fa_loading_corr_F2": float(corr_F2),
        "taste_topk_jaccard_mean": float(np.mean(overlaps)) if overlaps else np.nan,
        "taste_topk_jaccard_list": overlaps,
    }
    return stability

In [42]:
stability_eval = evaluate_model_stability(recommender.df)
stability_eval

{'fa_loading_corr_F1': 1.0,
 'fa_loading_corr_F2': 1.0,
 'taste_topk_jaccard_mean': 1.0,
 'taste_topk_jaccard_list': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0]}

# 저노출 제품 해결


    기본 추천 로직에서 거의 노출되지 않았던 상품들 중,
    사용자가 선택하거나 시스템이 추천한 위스키와
    FA taste space 상에서 동일/유사한 위치에 놓인 제품들을
    별도 슬롯(Something else with similar taste)으로 노출해
    롱테일 상품의 노출을 보완했다.

 📎 결과     

      •	기본 추천만 쓰면 약 절반(48%) 의 위스키만 고객에게 보인다.
      •	각 추천 결과에 “Something else?”라는 롱테일 슬롯 7칸을 붙이면,
        전체의 **58%**까지 노출되면서,
        그동안 한 번도 안 보이던 롱테일 제품 중 **약 18%**가 새로 노출된다.

In [58]:
def build_something_else_slot(
    recommender,
    main_result,
    under_exposed,
    neighbor_k=50,
    slot_size=7,
    taste_key=None,
):
    """
    'something else?' 슬롯용 long-tail 후보 생성

    - main_result:
        * recommender.recommend(...) 결과 dict 이거나
        * taste 추천 DataFrame 여도 됨.
    - under_exposed: 아직 한 번도 노출되지 않은 상품 name 리스트
    """

    # 1) taste_df 가져오기 (dict / df 모두 허용)
    if isinstance(main_result, dict):
        # 우선 호출하는 쪽에서 taste_key를 넘겨줬으면 최우선 사용
        cand_keys = []
        if taste_key is not None:
            cand_keys.append(taste_key)
        # 기본 후보 키들
        cand_keys.extend(["taste_based_main", "taste_only_main"])

        taste_df = None
        for k in cand_keys:
            if k in main_result:
                taste_df = main_result[k]
                break

        if taste_df is None:
            # taste 기반 추천이 없으면 슬롯도 만들 수 없음
            return pd.DataFrame()
    else:
        # 이미 DataFrame 이면 그대로 사용
        taste_df = main_result

    if taste_df is None or taste_df.empty:
        return pd.DataFrame()

    # 2) seed 후보: taste 추천 상위 N개 이름
    seed_candidates = taste_df["name"].dropna().unique().tolist()
    if not seed_candidates:
        return pd.DataFrame()

    long_tail_candidates = []

    # 3) 각 seed 주변에서 under_exposed 친구들 찾기
    for seed in seed_candidates:
        try:
            idx = recommender._get_index_by_name(seed)
        except ValueError:
            continue

        # seed의 FA 좌표
        if idx not in recommender.df_taste.index:
            continue
        point = recommender.df_taste.loc[idx, ["FA1", "FA2"]].values

        neigh_df = recommender._knn_taste_point(point, neighbor_k)

        # under_exposed 에만 포함된 애들만 남기기
        mask = neigh_df["name"].isin(under_exposed)
        neigh_df = neigh_df.loc[mask].copy()
        neigh_df["seed_name"] = seed

        long_tail_candidates.append(neigh_df)

    if not long_tail_candidates:
        return pd.DataFrame()

    cand_df = pd.concat(long_tail_candidates, ignore_index=True)

    # 4) taste_distance + seed_name 기준으로 정렬 후 상위 slot_size 개
    cand_df = cand_df.sort_values(["taste_distance", "seed_name"]).reset_index(drop=True)

    return cand_df.head(slot_size)

In [59]:
main_result = recommender.recommend(
    product_list=["Old Perth Double Sherry Wood Blended Malt", "Hakushu 25 Year Old"],
    top_k_taste=20,
    top_k_meta=20,
)

something_else_df = build_something_else_slot(
    recommender=recommender,
    main_result=main_result,
    under_exposed=under_exposed,
    neighbor_k=50,
    slot_size=7,
)

display(
    something_else_df[["name", "seed_name", "taste_distance", "whisky_type", "region","style_body","style_richness","style_smoke","style_sweetness"]].head(10)
)


,name,seed_name,taste_distance,whisky_type,region,style_body,style_richness,style_smoke,style_sweetness
0,Hakushu 18 Year Old Peated Malt100th Anniversary,Benriach The Smoky Ten10 Year Old,0.0,single malt,unknown_japan,4.0,2.0,2.0,3.0
1,Hakushu 18 Year Old Peated Malt100th Anniversary,Blended Malt SRV58 Year Old Thompson Bros,0.0,single malt,unknown_japan,4.0,2.0,2.0,3.0
2,Hibiki Harmony100th Anniversary,Chivas Regal 18 Year OldGift Box,0.0,blended,unknown_japan,3.0,4.0,1.0,3.0
3,Hibiki 21 Year Old Mizunara100th Anniversary,Chivas Regal 18 Year OldGift Box,0.0,blended,unknown_japan,3.0,4.0,1.0,3.0
4,Hibiki 21 Year Old,Chivas Regal 18 Year OldGift Box,0.0,blended,unknown_japan,3.0,4.0,1.0,3.0
5,Hibiki Harmony100th Anniversary,J&B 15 Year Old ReserveLitre,0.0,blended,unknown_japan,3.0,4.0,1.0,3.0
6,Hibiki 21 Year Old Mizunara100th Anniversary,J&B 15 Year Old ReserveLitre,0.0,blended,unknown_japan,3.0,4.0,1.0,3.0


In [60]:
def evaluate_coverage_with_something_else(
    recommender,
    build_something_else_slot,
    n_style_seeds=50,
    n_nostyle_seeds=30,
    n_survey_seeds=20,
    top_k_taste=20,
    top_k_meta=20,
    neighbor_k=50,
    slot_size=7,
    random_state=42,
):
    rng = np.random.default_rng(random_state)
    df = recommender.df

    # -----------------------------
    # 0) 기존 baseline 커버리지
    # -----------------------------
    base_eval = evaluate_business_exposure(
        recommender,
        n_style_seeds=n_style_seeds,
        n_nostyle_seeds=n_nostyle_seeds,
        n_survey_seeds=n_survey_seeds,
        top_k_taste=top_k_taste,
        top_k_meta=top_k_meta,
        random_state=random_state,
    )
    base_coverage = base_eval["coverage_ratio"]
    base_exposure_df = base_eval["exposure_df"]

    all_items = set(df["name"].unique())
    appeared_items = set(base_exposure_df["name"].unique())
    under_exposed = sorted(all_items - appeared_items)

    # -----------------------------
    # 1) 슬롯 포함해서 새로 시뮬레이션
    # -----------------------------
    all_reco_names = []

    # --- A) 스타일 있는 제품 seed ---
    style_seeds = df[df["style_missing"] == 0]["name"].dropna().unique()
    style_seeds = rng.choice(
        style_seeds, size=min(n_style_seeds, len(style_seeds)), replace=False
    )

    for name in style_seeds:
        res = recommender.recommend(
            product_list=[name],
            top_k_taste=top_k_taste,
            top_k_meta=top_k_meta,
        )

        # 기본 추천
        for key in ["taste_based_main", "meta_based_main"]:
            if key in res and isinstance(res[key], pd.DataFrame):
                all_reco_names.extend(res[key]["name"].tolist())

        # something else 슬롯 (taste_based_main 기준)
        if "taste_based_main" in res and not res["taste_based_main"].empty and under_exposed:
            slot_df = build_something_else_slot(
                recommender=recommender,
                main_result=res,
                under_exposed=under_exposed,
                neighbor_k=neighbor_k,
                slot_size=slot_size,
                taste_key="taste_based_main",
            )
            if not slot_df.empty:
                all_reco_names.extend(slot_df["name"].tolist())

    # --- B) 스타일 없는 제품 seed ---
    nostyle_seeds = df[df["style_missing"] == 1]["name"].dropna().unique()
    if len(nostyle_seeds) > 0:
        nostyle_seeds = rng.choice(
            nostyle_seeds, size=min(n_nostyle_seeds, len(nostyle_seeds)), replace=False
        )
        for name in nostyle_seeds:
            res = recommender.recommend(
                product_list=[name],
                top_k_meta=top_k_meta,
            )
            if "meta_based_main" in res and isinstance(res["meta_based_main"], pd.DataFrame):
                all_reco_names.extend(res["meta_based_main"]["name"].tolist())
            # B는 taste space 없으니 슬롯은 생략

    # --- C) 설문 seed ---
    body_levels = ["light", "medium", "full"]
    rich_levels = ["lean", "round", "rich"]
    smoke_levels = ["non", "light-smoky", "smoky"]
    sweet_levels = ["dry", "balanced", "sweet"]

    for _ in range(n_survey_seeds):
        res = recommender.recommend(
            body_level=rng.choice(body_levels),
            rich_level=rng.choice(rich_levels),
            smoke_level=rng.choice(smoke_levels),
            sweet_level=rng.choice(sweet_levels),
            selected_families=None,
            top_k_taste=top_k_taste,
            top_k_meta=top_k_meta,
        )

        for key in ["taste_only_main", "meta_expansion"]:
            if key in res and isinstance(res[key], pd.DataFrame):
                all_reco_names.extend(res[key]["name"].tolist())

        # 설문에서도 taste_only_main 기준으로 something else 슬롯
        if "taste_only_main" in res and not res["taste_only_main"].empty and under_exposed:
            slot_df = build_something_else_slot(
                recommender=recommender,
                main_result=res,
                under_exposed=under_exposed,
                neighbor_k=neighbor_k,
                slot_size=slot_size,
                taste_key="taste_only_main",
            )
            if not slot_df.empty:
                all_reco_names.extend(slot_df["name"].tolist())

    # -----------------------------
    # 2) 새로운 커버리지 계산
    # -----------------------------
    exposure_counts_new = (
        pd.Series(all_reco_names, name="name").value_counts().rename("exposure_count")
    )
    exposure_df_new = (
        exposure_counts_new.to_frame()
        .merge(df[["name"]], left_index=True, right_on="name", how="left")
    )

    total_items = df["name"].nunique()
    covered_items_new = exposure_counts_new.index.nunique()
    new_coverage = covered_items_new / total_items

    newly_covered_under = set(exposure_counts_new.index) & set(under_exposed)
    under_coverage_ratio = len(newly_covered_under) / max(len(under_exposed), 1)

    return {
        "base_coverage": base_coverage,
        "new_coverage": new_coverage,
        "delta_coverage": new_coverage - base_coverage,
        "under_exposed_size": len(under_exposed),
        "under_coverage_ratio": under_coverage_ratio,
        "base_exposure_df": base_exposure_df,
        "new_exposure_df": exposure_df_new,
    }

In [61]:
slot_eval = evaluate_coverage_with_something_else(
    recommender=recommender,
    build_something_else_slot=build_something_else_slot,
    n_style_seeds=50,
    n_nostyle_seeds=30,
    n_survey_seeds=20,
    top_k_taste=20,
    top_k_meta=20,
    neighbor_k=50,
    slot_size=7,
    random_state=42,
)

print("기존 coverage :", slot_eval["base_coverage"])
print("슬롯 포함 coverage :", slot_eval["new_coverage"])
print("증가분 delta      :", slot_eval["delta_coverage"])
print("under_exposed 개수:", slot_eval["under_exposed_size"])
print("under_exposed 중 이번에 노출된 비율:", slot_eval["under_coverage_ratio"])

기존 coverage : 0.48344370860927155
슬롯 포함 coverage : 0.5780510879848628
증가분 delta      : 0.09460737937559127
under_exposed 개수: 546
under_exposed 중 이번에 노출된 비율: 0.18315018315018314
